In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Concatenate
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow import cast,float32
from keras import backend as kb
from statistics import mean, stdev
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from tensorflow.keras import Input, Model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.svm import SVR

2023-04-09 01:17:33.500347: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
class Preprocessing():
    def __init__(self,k_size=6):
        self.k_size = k_size
        kmers = self.generate_kmers("",self.k_size)
        self.vectorizer = CountVectorizer(vocabulary = kmers)
        self.seqs = []
    
    def generate_kmers(self,current_kmer,current_depth):
        if current_depth == 1:
            return [current_kmer+"a",current_kmer+"t",current_kmer+"c",current_kmer+"g"]
        else:
            ret = self.generate_kmers(current_kmer+"a",current_depth-1)
            for nt in ['t','c','g']:
                ret += self.generate_kmers(current_kmer+nt,current_depth-1)
            return ret
    
    def generate_kmer_multiple(self,seqlist,k):
        kmer_list = []
        n = -1
        for seq in seqlist:
            kmer_list.append(self.generate_kmer_single(str(seq),k))
        return kmer_list
    
    def generate_kmer_single(self,seq,k):
        kmer = ""
        for i in range(0,len(seq)-k,1):
            kmer += seq[i:i+k]+" "
        return kmer[:-1]
    
    def CountKmers(self,seqs):
        if type(seqs) in [type([]),type(pd.core.series.Series([1]))]:
            kmer = self.generate_kmer_multiple(seqs, self.k_size)
            transformed_X = self.vectorizer.transform(kmer).toarray()
            return transformed_X
        else:
            raise ValueError("""Invalid 'seqs' format.
            Expected formats are 'list' or 'pandas.core.series.Series'.""")
            
    def ReadFASTA(self,filename,as_pd=True):
        if filename.split(".")[-1] not in ["fasta","fna","fa"]:
            raise ValueError('Invalid file format. Expected formats are ["fasta","fna","fa"].')
        file_handle = open(filename,"r")
        seqs = []
        seqid = []
        tmp_seq = ""
        for line in file_handle:
            if (line[0] == ">"):
                if tmp_seq != "":
                    seqs.append(tmp_seq)
                seqid.append(line.split("\n")[0][1:])
                tmp_seq = ""
            else:
                tmp_seq+=line.split("\n")[0]
        seqs.append(tmp_seq)
        file_handle.close()
        if as_pd:
            fasta = {}
            for i in range(len(seqs)):
                fasta[seqid[i]] = seqs[i]
            return pd.DataFrame(fasta,index=["sequence"]).transpose()["sequence"]
        else:
            return seqs, seqid

In [5]:
def split_test_train(da,multiplicand):
    pp = Preprocessing()
    for i in range(0,5,1):
        X_test = da["sequence"].loc[i*multiplicand:(i+1)*multiplicand]
        Y_test = da["copy_number"].loc[i*multiplicand:(i+1)*multiplicand]
        X_train = pd.concat([da["sequence"].loc[0:i*multiplicand],
                             da["sequence"].loc[(i+1)*multiplicand:]],axis = 0)
        Y_train = pd.concat([da["copy_number"].loc[0:i*multiplicand],
                             da["copy_number"].loc[(i+1)*multiplicand:]],axis = 0)
        X_train = pp.CountKmers(seqs=X_train)
        X_test = pp.CountKmers(seqs=X_test)
        pd.DataFrame(X_train).to_csv("datasets/splits/X_train_{}.csv".format(str(i)),index=False)
        pd.DataFrame(Y_train).to_csv("datasets/splits/Y_train_{}.csv".format(str(i)),index=False)
        pd.DataFrame(X_test).to_csv("datasets/splits/X_test_{}.csv".format(str(i)),index=False)
        pd.DataFrame(Y_test).to_csv("datasets/splits/Y_test_{}.csv".format(str(i)),index=False)
    return

In [6]:
da = pd.read_csv("datasets/full_length_reads.csv")
multiplicand = int(da.shape[0]*0.2)
# split_test_train(da,multiplicand)

In [7]:
pp = Preprocessing()
da1 = pd.read_csv("datasets/full_length_testdata.filtered.csv")
X0 = da["sequence"]
Y0 = da['copy_number']
X1 = da1["sequence"]
Y1 = da1['copy_number']
X0 = pp.CountKmers(X0)
X1 = pp.CountKmers(X1)

In [8]:
pd.DataFrame(X0).to_csv("4.final_test/datasets/X_final_train.csv",index=False)
pd.DataFrame(Y0).to_csv("4.final_test/datasets/Y_final_train.csv",index=False)
pd.DataFrame(X1).to_csv("4.final_test/datasets/X_final_test.csv",index=False)
pd.DataFrame(Y1).to_csv("4.final_test/datasets/Y_final_test.csv",index=False)